<a href="https://colab.research.google.com/github/cartmarsh/DataAnalysis/blob/main/Toxic_comment_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Outline

1. Get the tweets with tweepy
2. load the model
3. feed it the tweets

4. Make some Data Analysis

In [5]:
!pip install tweepy==4.1.0
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install wordcloud
!pip install better_profanity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 KB 3.5 MB/s eta 0:00:00


In [8]:
import re 

import numpy as np

import tweepy 

from tweepy import OAuthHandler 

from textblob import TextBlob 

import matplotlib.pyplot as plt

import pandas as pd

from wordcloud import WordCloud

from better_profanity import profanity

In [13]:
consumer_key = 'uS3mcCvexB7CLcalnKTcEgzqo'
consumer_secret = 'ensBvZme0LUyTEqYq8XTxtjrNPyzms5slM1GaxmS6k7yxzzEy2' 
access_token = '840693273345822720-9lil2py7RXN0gkuvOyOOCcnFjAUJ7CU'
access_token_secret = '4wOCEbHoLW8cXFJWXw4QYvUp4t5MvNQTvBC2UbFUZ9hQq'

In [14]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = tweepy.API(auth)

In [15]:
tweepy.__version__

'4.1.0'

In [16]:
# Input a query from the user

query = input("Please enter your topic / person of interest: ")

# In this case, we will input the query as 'Elon Musk'

# Filter the query to remove retweets

filtered = query + "-filter:retweets"

# Generate the latest tweets on the given query 

tweets = tweepy.Cursor(api.search_tweets, 
                           q=filtered,
                           lang="en").items(100)

# Create a list of the tweets, the users, and their location

list1 = [[tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]

Please enter your topic / person of interest: Pyramids


In [17]:
# Convert the list into a dataframe

df = pd.DataFrame(data=list1, 
                    columns=['tweets','user', "location"])
# Convert only the tweets into a list

tweet_list = df.tweets.to_list()

In [49]:
tweet_list[0]

'@minnySODAguy Humans did not solely build the pyramids.. idk who did or helped but I can’t fathom humans at the tim… https://t.co/pEjia2DxmQ'

We can improve the stopwords here by donwloading a library

In [19]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    r = tweet.lower()
    r = profanity.censor(r)
    r = re.sub("'", "", r) # This is to avoid removing contractions in english
    r = re.sub("@[A-Za-z0-9_]+","", r)
    r = re.sub("#[A-Za-z0-9_]+","", r)
    r = re.sub(r'http\S+', '', r)
    r = re.sub('[()!?]', ' ', r)
    r = re.sub('\[.*?\]',' ', r)
    r = re.sub("[^a-z0-9]"," ", r)
    r = r.split()
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    r = [w for w in r if not w in stopwords]
    r = " ".join(word for word in r)
    return r

In [20]:
cleaned = [clean_tweet(tw) for tw in tweet_list]
cleaned[:10]

['humans did not solely build pyramids idk who did or helped but i can t fathom humans at tim',
 'personally i accept levitation as method assembling giant stones they knew what caused gravi',
 'ancient egyptian pyramids have stood thousands years are among worlds most enduring monuments',
 'live podcast nibiru episode 213 dark skies news information',
 'step pyramid saqqara another great site you will experience with us come travel with us',
 'we are thrilled that wednesday february 22nd we will have pyramids food truck here 5pm 7 30pm we h',
 'new audiobook addition easy nevada pyramid s curse by georgette kaplan audiobook review',
 'case you missed it michelle easy nevada pyramid s curse by georgette kaplan is adrenaline fueled',
 'how did majestic pyramids ancient egypt appear their original form',
 'josefina en las pyramids']

Using

# Huggingface sentiment model

Using huggingface sentiment analysis model

In [21]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)





In [44]:
def tweetPred(tweet):
  text = tweet
  encoded_input = tokenizer(tweet, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  # # TF
  # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
  # model.save_pretrained(MODEL)
  # text = "Covid cases are increasing fast!"
  # encoded_input = tokenizer(text, return_tensors='tf')
  # output = model(encoded_input)
  # scores = output[0][0].numpy()
  # scores = softmax(scores)
  # Print labels and scores
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  print(tweet)

  rankingArray = []

  for i in range(scores.shape[0]):
      l = config.id2label[ranking[i]]
      s = scores[ranking[i]]
      rankingArray.append(scores[ranking[i]])
      print(f"{i+1}) {l} {np.round(float(s), 4)}")
  return [tweet, rankingArray]

In [45]:
type(cleaned)

list

In [75]:
tweetPreds = map(tweetPred, cleaned)

In [47]:
type(tweetPreds)

map

In [76]:
import numpy as np

def to_numpy_array(tweetPreds):
    result = []
    for tweet, rankingArray in tweetPreds:
        row = [rankingArray[0], rankingArray[2], rankingArray[1]]
        print(row)
        result.append(row)
    return np.array(result)

In [ ]:
np_tweetPreds = to_numpy_array(tweetPreds)

In [79]:
np_tweetPreds.shape

(100, 3)

In [60]:
print(np_tweetPreds)


[]


In [48]:
print(list(tweetPreds))

humans did not solely build pyramids idk who did or helped but i can t fathom humans at tim
1) negative 0.8221
2) neutral 0.1663
3) positive 0.0116
personally i accept levitation as method assembling giant stones they knew what caused gravi
1) neutral 0.6944
2) negative 0.2077
3) positive 0.0979
ancient egyptian pyramids have stood thousands years are among worlds most enduring monuments
1) neutral 0.5323
2) positive 0.4502
3) negative 0.0175
live podcast nibiru episode 213 dark skies news information
1) neutral 0.9419
2) positive 0.0482
3) negative 0.01
step pyramid saqqara another great site you will experience with us come travel with us
1) positive 0.9774
2) neutral 0.0207
3) negative 0.0019
we are thrilled that wednesday february 22nd we will have pyramids food truck here 5pm 7 30pm we h
1) positive 0.9892
2) neutral 0.0096
3) negative 0.0012
new audiobook addition easy nevada pyramid s curse by georgette kaplan audiobook review
1) neutral 0.7628
2) positive 0.2137
3) negative 0.0

Load the model

In [ ]:
import torch

In [ ]:
!pwd


In [ ]:
!ls

In [ ]:
model = torch.load("./checkpointRNN_toxic_comment_classifier.ckpt")

In [ ]:
!pip install pytorch_lightning

In [ ]:
import pytorch_lightning as pl

Prepare the data for prediction

In [ ]:
from torchtext.data.utils import get_tokenizer

In [ ]:
tokenizer = get_tokenizer("basic_english")

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
comments_token = [tokenizer(comment) for comment in cleaned]

In [ ]:
comments_token = pd.DataFrame(comments_token)

In [ ]:
comments_token[:10]

In [ ]:
VOCAB_SIZE = 1500

In [ ]:
## for words that are unknown to the vocab
unk_token = ""

## to pad sentences, because we need the same length for RNNs
pad_token = ""

In [ ]:
vocab = build_vocab_from_iterator(comments_token, specials=[unk_token, pad_token],
                                  max_tokens=VOCAB_SIZE)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
class JigsawDataset(Dataset):
  def __init__(self, df, is_test=False):
    self.df = df
    self.is_test = is_test
    
  
  def __getitem__(self, index):
    comment_text = self.df.comment_text.values[index]
    comment_tokens = pad_tokens(tokenizer(comment_text))
    input = torch.tensor(vocab.lookup_indices(comment_tokens))
    if self.is_test:
      target = torch.tensor([0,0,0,0,0,0]).float()
    else:
      target = torch.tensor(self.df[target_cols].values[index]).float()
    return input, target

  def __len__(self):
    return len(self.df)